# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_sig, y_test_sig = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_tanh, y_test_tanh = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_elu, y_test_elu = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)
X_train_sig= sc.fit_transform(X_train)
X_test_sig= sc.transform(X_test)
X_train_tanh= sc.fit_transform(X_train)
X_test_tanh= sc.transform(X_test)
X_train_elu= sc.fit_transform(X_train)
X_test_elu = sc.transform(X_test)


In [8]:
np.shape(X_train)
np.shape(X_train_sig)
np.shape(X_train_tanh)
np.shape(X_train_elu)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()#RELU
ann1 = tf.keras.models.Sequential()#SIGMÓIDE
ann2 = tf.keras.models.Sequential()#TANH
ann3 = tf.keras.models.Sequential()#ELU = 'EXPONENCIAL LINEAR UNITY'

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))#RELU
ann1.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))#SIGMOIDE
ann2.add(tf.keras.layers.Dense(units=6, activation='tanh'))#TANH
ann3.add(tf.keras.layers.Dense(units=6, activation='elu'))#ELU

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann1.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))#SIGMOIDE
ann2.add(tf.keras.layers.Dense(units=6, activation='tanh'))#TANH
ann3.add(tf.keras.layers.Dense(units=6, activation='elu'))#ELU

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))#1-RELU, 2-RELU,3-SIGMOID
ann1.add(tf.keras.layers.Dense(units=1, activation='tanh'))#1-SIGMOID, 2-SIGMOID,3-TANH
ann2.add(tf.keras.layers.Dense(units=1, activation='elu'))#1-TANH, 2-TANH,3-ELU
ann3.add(tf.keras.layers.Dense(units=1, activation='relu'))#1-ELU, 2-ELU,3-RELU

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann3.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)


Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4942 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4521 - accuracy: 0.7968
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8030
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4201 - accuracy: 0.8148
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4104 - accuracy: 0.8254
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4033 - accuracy: 0.8292
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3973 - accuracy: 0.8298
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3925 - accuracy: 0.8290
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3874 - accuracy: 0.8306
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3836 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [15]:
y_pred = ann.predict(X_test)
y_pred

array([[0.20329666],
       [0.29358667],
       [0.10408321],
       ...,
       [0.29254055],
       [0.11383837],
       [0.2291317 ]], dtype=float32)

In [16]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1523   72]
 [ 208  197]]


In [ ]:
SIGMOIDE

In [71]:
ann1.fit(X_train_sig, y_train_sig, batch_size = 32, epochs = 100)



Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5980 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4869 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4553 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4443 - accuracy: 0.7964
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4416 - accuracy: 0.7979
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4413 - accuracy: 0.8001
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4394 - accuracy: 0.8041
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4550 - accuracy: 0.8060
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4362 - accuracy: 0.8026
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4390 - accura

In [72]:
y_pred1 = ann1.predict(X_test_sig)
y_pred1

array([[0.29144987],
       [0.2835193 ],
       [0.23600927],
       ...,
       [0.18785611],
       [0.18071231],
       [0.31172687]], dtype=float32)

In [73]:
y_pred1 = (y_pred1 > 0.5)
y_pred1

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [74]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_sig, y_pred1)
print(cm)

[[1560   35]
 [ 283  122]]


In [ ]:
TANH

In [75]:
ann2.fit(X_train_tanh, y_train_tanh, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 1.3473 - accuracy: 0.7639
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 1.0334 - accuracy: 0.7624
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8685 - accuracy: 0.7656
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7089 - accuracy: 0.7491
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6098 - accuracy: 0.7631
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6203 - accuracy: 0.7398
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5836 - accuracy: 0.7570
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5412 - accuracy: 0.7666
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5209 - accuracy: 0.7731
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5052 - accura

In [76]:
y_pred2 = ann2.predict(X_test_tanh)
y_pred2

array([[0.36174196],
       [0.28622934],
       [0.16294275],
       ...,
       [0.14369631],
       [0.17331687],
       [0.1860709 ]], dtype=float32)

In [77]:
y_pred2 = (y_pred2 > 0.5)
y_pred2

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [78]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_tanh, y_pred2)
print(cm)

[[1563   32]
 [ 244  161]]


In [ ]:
ELU

In [79]:
ann3.fit(X_train_elu, y_train_elu, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 2.4898 - accuracy: 0.6532
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5550 - accuracy: 0.7054
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 1.1408 - accuracy: 0.7254
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7669 - accuracy: 0.7412
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5939 - accuracy: 0.7624
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5434 - accuracy: 0.7688
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5208 - accuracy: 0.7736
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4958 - accuracy: 0.7772
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4877 - accuracy: 0.7820
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4813 - accura

In [80]:
y_pred3 = ann3.predict(X_test_elu)
y_pred3

array([[0.26318017],
       [0.38292658],
       [0.14367607],
       ...,
       [0.21477565],
       [0.17962343],
       [0.30819544]], dtype=float32)

In [81]:
y_pred3 = (y_pred3 > 0.5)
y_pred3

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [83]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_elu, y_pred3)
print(cm)

[[1541   54]
 [ 243  162]]


In [27]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
#accuracies1 = cross_val_score(estimator = classifier, X = X_train_sig, y = y_train_sig, cv = 10, n_jobs = -1)
#accuracies2 = cross_val_score(estimator = classifier, X = X_train_tanh, y = y_train_tanh, cv = 10, n_jobs = -1)
#accuracies3 = cross_val_score(estimator = classifier, X = X_train_elu, y = y_train_elu, cv = 10, n_jobs = -1)
#mean1 = accuracies1.mean()
#mean2 = accuracies2.mean()
#mean3 = accuracies3.mean()
#variance1 = accuracies1.std()
#variance2 = accuracies2.std()
#variance3 = accuracies3.std()
print(accuracies)
print(mean)
print(variance)


[nan nan nan nan nan nan nan nan nan nan]
nan
nan


In [16]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
accuracies1 = cross_val_score(estimator = classifier, X = X_train_sig, y = y_train_sig, cv = 10, n_jobs = -1)
accuracies2 = cross_val_score(estimator = classifier, X = X_train_tanh, y = y_train_tanh, cv = 10, n_jobs = -1)
accuracies3 = cross_val_score(estimator = classifier, X = X_train_elu, y = y_train_elu, cv = 10, n_jobs = -1)
mean = accuracies.mean()
mean1 = accuracies1.mean()
mean2 = accuracies2.mean()
mean3 = accuracies3.mean()
variance = accuracies.std()
variance1 = accuracies1.std()
variance2 = accuracies2.std()
variance3 = accuracies3.std()

In [17]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [10, 25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_


grid_search1 = grid_search.fit(X_train_sig, y_train_sig)

best_parameters1 = grid_search1.best_params_
best_accuracy1 = grid_search1.best_score_


grid_search2 = grid_search.fit(X_train_tanh, y_train_tanh)

best_parameters2 = grid_search2.best_params_
best_accuracy2 = grid_search2.best_score_

grid_search3 = grid_search.fit(X_train_elu, y_train_elu)

best_parameters3 = grid_search3.best_params_
best_accuracy3 = grid_search3.best_score_

print('melhores parametros ReLU: '+str(best_parameters)+'melhor exatidão ReLU: '+str(best_accuracy))
print('melhores parametros Sig: '+str(best_parameters1)+'melhor exatidão Sig: '+str(best_accuracy1))
print('melhores parametros Tanh: '+str(best_parameters2)+'melhor exatidão Tanh: '+str(best_accuracy2))
print('melhores parametros ElU: '+str(best_parameters3)+'melhor exatidão ElU: '+str(best_accuracy3))


artition for these parameters will be set to nan. Details: 
ValueError: in user code:

    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **k

ValueError: in user code:

    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\pc home jan 2020\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\pc home jan 2020\anaconda3\lib\site-pack

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)